In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [2]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.tail()

,Model_name,input_shape,ACET,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
1525,basic_model_64_15,"(1, 64, 64, 3)",185.188635,64,12288,37300460,5760,48100,10696805810,27,15,5,1,3050376,5309448448
1526,basic_model_32_44,"(1, 32, 32, 3)",182.426993,32,3072,32517012,14352,513000,23311729169,74,44,20,1,5247440,11618736064
1527,basic_model_112_17,"(1, 112, 112, 3)",167.433424,112,37632,15046344,9488,1210,51190739064,33,17,8,1,25374078,25571214384
1528,basic_model_75_37,"(1, 75, 75, 3)",159.081569,75,16875,18939872,9264,80500,26945702640,64,37,19,1,9258839,13449790920
1529,basic_model_32_23,"(1, 32, 32, 3)",135.091388,32,3072,16689184,9632,491530,34211079000,41,23,14,1,6237204,17086005248


In [110]:
# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.tail()

,Model_name,input_shape,ACET,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
102,EfficientNet-B0,"(512, 512, 3)",69.488341,512,786432,3965532,84032,1281000,4085726980,233,81,49,1,117635172,80094501376
103,EfficientNet-B0,"(720, 720, 3)",136.853771,720,1555200,3965532,84032,1281000,8183072596,233,81,49,1,233556068,161236938752
104,EfficientNet-B4,"(90, 90, 3)",30.377035,90,24300,17423416,250400,1793000,595640786,470,160,96,1,10284484,12812090224
105,EfficientNet-B4,"(512, 512, 3)",189.856248,512,786432,17423416,250400,1793000,15877844034,470,160,96,1,305708296,366198974816
106,EfficientNet-B4,"(720, 720, 3)",372.265183,720,1555200,17423416,250400,1793000,31863479154,470,160,96,1,607426440,737567732576


In [5]:
train_dataset.pop('input_width')
train_dataset.pop('nb_params_fc')
train_dataset.pop('nb_params_bn')
train_dataset.pop('nb_bn_layers')
train_dataset.pop('nb_fc_layers')
train_dataset.pop('nb_conv_layers')

0       28
1       28
2       28
3       28
4       28
        ..
1525    15
1526    44
1527    17
1528    37
1529    23
Name: nb_conv_layers, Length: 1530, dtype: int64

In [111]:
test_dataset.pop('input_width')
test_dataset.pop('nb_params_fc')
test_dataset.pop('nb_params_bn')
test_dataset.pop('nb_bn_layers')
test_dataset.pop('nb_fc_layers')
test_dataset.pop('nb_conv_layers')

0      160
1      160
2      168
3      168
4      200
      ... 
102     81
103     81
104    160
105    160
106    160
Name: nb_conv_layers, Length: 107, dtype: int64

In [7]:
Y = train_dataset.pop('ACET')
X = train_dataset
Y_f = Y
X_f = X

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [9]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [10]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [11]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [112]:
Y_e = test_dataset.pop('ACET')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [92]:
params = {
    # Parameters that we are going to tune.
    'max_depth':20,
    'min_child_weight': 5,
    'eta':.01,
    'subsample': .3,
    'colsample_bytree': 1.,
    'gamma' : 1.5,
    # Other parameters
    'objective':'reg:squarederror',
}

In [93]:
params['eval_metric'] = "mae"

In [94]:
print(params)

{'max_depth': 20, 'min_child_weight': 5, 'eta': 0.01, 'subsample': 0.3, 'colsample_bytree': 1.0, 'gamma': 1.5, 'objective': 'reg:squarederror', 'eval_metric': 'mae'}


In [95]:
num_boost_round = 271
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
)

In [96]:
loaded_model = best_model

In [113]:
y_pred = loaded_model.predict(dtest_e)
np_test = Y_e.values
y_test = Y_e.values

In [114]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [115]:
# Model evaluation on test dataset
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error :', median_absolute_error(y_test, y_pred))

Explained variance :  0.9872492759112206
R2_value :  0.9863780345977018
Mean absolute error : 21.693975333507673
Root Mean squared error : 41.2082988143235
Median absolute error : 9.294283371874997


In [116]:
# Calculate mean absolute percentage error (MAPE)
np_test = y_test
errors = abs(y_pred-np_test)
mape = 100 * (errors / np_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE : ',max(mape))
print('Mape:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE :  40.6889536198578
Mape: 9.58 %.
Accuracy: 90.42 %.


In [117]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'real_exec_time', 'predicted_exec_time', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, np_test[i], y_pred[i],abs(y_pred[i]-np_test[i]), ((abs(y_pred[i]-np_test[i]))/np_test[i])*100])